In [ ]:
import geopandas as gpd # for geospatial data handling
import pandas as pd
#import osmnx # for handling data from OpenStreetMap (osm) with the help of networkX (nx)
import contextily as cx # for plotting
import matplotlib.pyplot as plt # for plotting
from pyproj import CRS # for more advanced CRS modifications and transformations
import xml.etree.ElementTree as ET
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import rasterio
import numpy as np

In [ ]:
from fastkml import kml
import geopandas as gpd

def kml_to_geodataframe(kml_file_path):
    # Read the KML file
    with open(kml_file_path, 'r', encoding='utf-8') as kml_file:
        kml_content = kml_file.read()

    # Create a KML object
    k = kml.KML()
    k.from_string(kml_content)

    # Assume the first feature is the Document
    doc = list(k.features())[0]

    # Initialize an empty list to hold geometries and properties
    data = []

    # Iterate through all placemarks in the Document
    for placemark in doc.features():
        geom = placemark.geometry
        name = placemark.name  # Optional: extract name
        
        # Check if the geometry is a LineString, adjust as needed
        if geom.geom_type == 'LineString':
            # If it's a LineString, proceed normally
            data.append({'geometry': geom, 'name': name})
        elif geom.geom_type == 'Polygon':
            # Optional: If it's a Polygon but expected to be a LineString, convert the exterior to a LineString
            exterior = geom.exterior
            line = LineString(exterior.coords)
            data.append({'geometry': line, 'name': name})
        else:
            # For other types, you might want to handle them differently or skip
            continue

    # Convert the list of dictionaries to a GeoDataFrame
    gdf = gpd.GeoDataFrame(data, columns=['geometry', 'name'])

    return gdf

# Convert KML to GeoDataFrame for each of your files
left_boundary_gdf = kml_to_geodataframe("Left_boundary_track.kml")
right_boundary_gdf = kml_to_geodataframe("Right_boundary_track.kml")

# Example usage of the GeoDataFrames
print(left_boundary_gdf.head())
print(right_boundary_gdf.head())


In [ ]:
left_boundary_gdf.geometry.head()

In [ ]:
left_boundary_gdf.plot()

In [ ]:
right_boundary_gdf.plot()

In [ ]:
import rasterio

# Path to your TIFF file
tif_path = 'track_map.tif'

# Open the TIFF file
with rasterio.open(tif_path) as src:
    # Print the raster size (width and height in pixels)
    print(f"Width: {src.width}, Height: {src.height}")
    
    # Print the bounding box of the raster
    print(f"Bounds: {src.bounds}")
    
    # Print the Coordinate Reference System (CRS)
    print(f"CRS: {src.crs}")
    
    # Print the affine transform (pixel to spatial coordinate transformation)
    print(f"Transform: {src.transform}")
    
    # Example of defining a window: Read a 100x100 pixel window starting from (0, 0)
    #window = rasterio.windows.Window(col_off=0, row_off=0, width=100, height=100)
    #data = src.read(window=window)
    
    # Print the shape of the data read from the window
    #print(f"Data shape (bands, rows, columns): {data.shape}")


In [ ]:
import rasterio
from rasterio.windows import from_bounds
from rasterio.transform import from_origin

# Open the original TIFF
with rasterio.open("track_map.tif") as src:
    # Your desired geographic coordinates
    right, bottom, left, top = 6.9526, 50.3224, 6.9308, 50.3383
    
    # Calculate the window position in pixel coordinates
    window = from_bounds(left, bottom, right, top, src.transform)
    
    # Read the data from the window
    data = src.read(window=window)
    
    # Define the new transform for the cutout
    new_transform = from_origin(left, top, src.res[0], src.res[1])
    
    # Update metadata for the output file
    out_meta = src.meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": data.shape[1],
        "width": data.shape[2],
        "transform": new_transform,
        "compress": "lzw"
    })

    # Write the data to a new TIFF file
    with rasterio.open("track_map_cut.tif", "w", **out_meta) as dest:
        dest.write(data)


In [ ]:
# Open the TIFF file
with rasterio.open('track_map_cut.tif') as src:
    # Read the RGB bands
    red = src.read(1)
    green = src.read(2)
    blue = src.read(3)

    # Stack the bands
    rgb = np.dstack((red, green, blue))

    # Plot the data
    plt.figure(figsize=(10, 10))
    plt.imshow(rgb)
    plt.title('Cut Section of the TIFF File')
    plt.show()


In [ ]:
import rasterio
import rasterio.plot
import matplotlib.pyplot as plt
import geopandas as gpd


# Open your TIFF file
with rasterio.open('track_map_cut.tif') as src:
    # Read the data
    data = src.read(1)  # Assuming it's a single-band TIFF
    
    # Get the bounds of the raster
    bounds = src.bounds
    
    # Create a figure and axis for plotting
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Display the raster basemap
    rasterio.plot.show(src, ax=ax, cmap='gray')  # Use cmap='gray' for single-band, remove for RGB

    # Overlay the GeoDataFrames
    left_boundary_gdf.plot(ax=ax, color='blue', linewidth=2, label='Left Boundary')
    right_boundary_gdf.plot(ax=ax, color='red', linewidth=2, label='Right Boundary')
    
    # Set the plot limits to the bounds of the raster
    ax.set_xlim(bounds.left, bounds.right)
    ax.set_ylim(bounds.bottom, bounds.top)
    
    # Optional: Add a legend and title
    ax.legend()
    ax.set_title('Basemap with LineString Boundaries')

    plt.show()


In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np

with rasterio.open('track_map_cut.tif') as src:
    # Read the RGB bands
    r = src.read(1)
    g = src.read(2)
    b = src.read(3)
    
    # Stack the bands
    rgb = np.dstack((r, g, b))
    
    # Save the full-color image
    plt.imsave('map_track_cut.png', rgb)


In [ ]:
# Convert KML to GeoDataFrame for each of your files
left_boundary_gdf = kml_to_geodataframe("Left_boundary_track.kml")
right_boundary_gdf = kml_to_geodataframe("Right_boundary_track.kml")

# Check if your GeoDataFrames have a CRS set
print(left_boundary_gdf.crs)
print(right_boundary_gdf.crs)

# If the output is None or not EPSG:4326, set/transform the CRS to EPSG:4326
if left_boundary_gdf.crs is None:
    left_boundary_gdf = left_boundary_gdf.set_crs("EPSG:4326")
else:
    left_boundary_gdf = left_boundary_gdf.to_crs("EPSG:4326")

if right_boundary_gdf.crs is None:
    right_boundary_gdf = right_boundary_gdf.set_crs("EPSG:4326")
else:
    right_boundary_gdf = right_boundary_gdf.to_crs("EPSG:4326")


from shapely.affinity import translate
import geopandas as gpd

# Define the shift values
shift_x = -0.000025 # Negative value for shifting to the left
shift_y = -0.00001  # No shift in latitude

# Function to apply the shift using shapely.affinity.translate
def shift_geometry(geometry, dx, dy):
    return translate(geometry, xoff=dx, yoff=dy)



# Apply the shift to each geometry in the GeoDataFrames
right_boundary_gdf['geometry'] = right_boundary_gdf['geometry'].apply(shift_geometry, args=(shift_x, shift_y))
left_boundary_gdf['geometry'] = left_boundary_gdf['geometry'].apply(shift_geometry, args=(shift_x, shift_y))



import folium

# Create a basic Folium map
m = folium.Map(location=[50.33, 6.94], zoom_start=13, tiles='CartoDB positron')


# Add the full-color ImageOverlay
folium.raster_layers.ImageOverlay(
    image='map_track_cut.png',
    bounds=[[50.3224, 6.9308], [50.3383, 6.9526]],  # Adjust bounds to your image
    opacity=1,  # Adjust as needed
).add_to(m)

# Add left boundary GeoDataFrame to the map
folium.GeoJson(
    left_boundary_gdf,
    name='Left Boundary',
    style_function=lambda feature: {
        'color': 'blue',
        'weight': 2,
        'opacity': 0.5
    }
).add_to(m)

# Add right boundary GeoDataFrame to the map
folium.GeoJson(
    right_boundary_gdf,
    name='Right Boundary',
    style_function=lambda feature: {
        'color': 'blue',
        'weight': 2,
        'opacity': 0.5
    }
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Display the map
m
